# Notes
## Camera Positioning
Seems that the estimations work best when the camera is position from the side or portrait in a reasonable length, centered around the hips.

In [2]:
import cv2
import mediapipe as mp
import time

mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

In [224]:
# TODO: check if OpenCV or MediaPipe/Tensorflow doesn't provide already
def calc_distance(landmark1, landmark2):
    xx = (landmark1.x - landmark2.x)**2
    yy = (landmark1.y - landmark2.y)**2
    zz = (landmark1.z - landmark2.z)**2
    
    return (xx + yy + zz)**(1/2)

In [238]:
file = "deadlifts1.mp4"

rep_frames = {
  "lat_pulldown1.mp4": [1, 118, 267, 398, 528, 663, 783, 899, 1026, 1143, 1290, 1492, 1651],
  "lat_pulldown2.mp4": [234, 350, 475, 601, 726, 864, 1011, 1159, 1304, 1451, 1670, 1825, 2049, 2278],
  "deadlift3.mp4": [77, 199, 303, 434, 566, 713, 852],
  "deadlift2.mp4": [226, 384, 492, 585, 723, 829, 963, 1094],
  "incline_db_press2.mp4": [149, 292, 416, 547, 682, 905, 1076, 1329, 1535],
  "tatwood_dl.mp4": [50, 128, 281],
  "tatwood_squat.mp4": [70, 260, 446, 650]
}

In [242]:
path = f"data/{file}"
cap = cv2.VideoCapture(path)
fps= cap.get(cv2.CAP_PROP_FPS)

if (cap.isOpened()== False): 
  print("Error opening video stream or file")

# TODO: In real time figure out which side is closer to the camera and use the appropriate wrist, elbow. Can always ask the user.
# TODO: Figure out the distance travelled.
# TODO: Identify individual repetitions.
# TODO: Figure out when the movement starts.
FOREARM_LENGTH_M = 0.24
forearm_length_im = 0
WRIST = mp_pose.PoseLandmark.LEFT_WRIST
ELBOW = mp_pose.PoseLandmark.LEFT_ELBOW
wrist_landmarks = []

cnt = 0

with mp_pose.Pose(
    min_detection_confidence = 0.7,
    min_tracking_confidence = 0.9
) as pose:
    while(cap.isOpened()):
        ret, frame = cap.read()

        if ret:
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False

            results = pose.process(image)

            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
            cv2.imshow(path, image)

            print(f"left wrist: {results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_WRIST].z}, ", end = '')
            print(f"right wrist: {results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_WRIST].z}")

            wrist_landmarks.append(results.pose_world_landmarks.landmark[WRIST])

            # Calculate the running average of forearm length in px
            forearm_length_im = (cnt*forearm_length_im + calc_distance(results.pose_world_landmarks.landmark[WRIST], results.pose_world_landmarks.landmark[ELBOW]))/(cnt + 1)

            key = cv2.waitKey(1)

            if key & 0xFF == ord('m'):
                print(cnt)

            if key & 0xFF == ord('q'):
                break

            cnt += 1
        else:
            break

cap.release()
cv2.destroyAllWindows()

left wrist: -0.5364753007888794, right wrist: -0.059740256518125534
left wrist: -0.4372139275074005, right wrist: -0.026686998084187508
left wrist: -0.43410980701446533, right wrist: -0.015308466739952564
left wrist: -0.4056971073150635, right wrist: 0.0023510828614234924
left wrist: -0.4010094404220581, right wrist: 0.005635555367916822
left wrist: -0.4372757375240326, right wrist: -0.007615335751324892
left wrist: -0.43422847986221313, right wrist: -0.014729315415024757
left wrist: -0.4445834457874298, right wrist: -0.011251410469412804
left wrist: -0.4705614447593689, right wrist: -0.014991267584264278
left wrist: -0.4780285954475403, right wrist: -0.016668494790792465
left wrist: -0.4803119897842407, right wrist: -0.017091970890760422
left wrist: -0.4778987765312195, right wrist: -0.014645125716924667
left wrist: -0.45059654116630554, right wrist: -0.010655143298208714
left wrist: -0.44245660305023193, right wrist: -0.0116340396925807
left wrist: -0.4344462752342224, right wrist: -

In [214]:
print(forearm_length_im)

0.23507431850831575


In [223]:
# TODO: Compare accuracy
# TODO: Check if the landmarks are visible enough
# TODO: Compute in real time
# TODO: Correct with the real forearm meassurements?
# TODO: Handle the last rep
# TODO: Identify reps
# TODO: Speed on concentric part only

for rep in range(len(rep_frames[file]) - 1):
  sum_im = 0
  frame_start = rep_frames[file][rep]
  frame_end = rep_frames[file][rep + 1] + 1

  for j in range(frame_start, frame_end):
    distance_im = calc_distance(wrist_landmarks[j-1], wrist_landmarks[j])
    sum_im += distance_im

  time_diff = (frame_end - frame_start) / fps
  speed = sum_im / time_diff
  print(speed)

# sum_m = sum_im*(FOREARM_LENGTH_M/forearm_length_im)
# print(sum_m/2)

0.2702166557601682
0.28278608575412933
0.25227381109912356
0.25262809378288803
0.22736990794453935
0.2437746955686146
